In [2]:
!echo 'Welcome to simple Souffle graph benchmarking.'
!echo 'Behold the power of The Machine:'
!echo "CPUs: $(nproc) / RAM: $(free -h | awk '/^Mem:/ {print $2}')"

Welcome to simple Souffle graph benchmarking.
Behold the power of The Machine:
CPUs: 32 / RAM: 125Gi


In [3]:
from IPython.core.magic import register_cell_magic
from IPython import get_ipython
import time
from logica.common import sqlite3_logica
import pandas

timing = {}

reports = []

@register_cell_magic
def loop(line, cell):
    global timing
    local_timing = {}
    ip = get_ipython()
    # Evaluate the line to get the list (e.g., "my_files")
    problem_name, iterator = ip.ev(line) 
    
    for item in iterator:
        # Inject 'item' into global namespace so the inner magic sees it
        ip.user_ns['loop_parameter'] = item 
        # Run the content as a new cell execution
        start_time = time.perf_counter()
        ip.run_cell(cell.replace('{loop_parameter}', item))
        end_time = time.perf_counter()
        elapsed = end_time - start_time
        timing[item] = elapsed
        local_timing[item] = elapsed
    report = (' === Timing for %s ===\n' % problem_name) + (
        sqlite3_logica.DataframeAsArtisticTable(pandas.DataFrame({'problem': list(local_timing.keys()),
                                                                  'time': list(local_timing.values())})))
    reports.append(report)
    print(report)
    


In [4]:
test_items = ['a', 'b', 'c']

In [5]:
%%loop ("Test Problem", test_items)
print("{loop_parameter}")

a
b
c
 === Timing for Test Problem ===
+---------+------------------------+
| problem | time                   |
+---------+------------------------+
| a       | 0.00022014000001036038 |
| b       | 0.00017433999994409533 |
| c       | 0.00016089000007468712 |
+---------+------------------------+


In [6]:
print(reports[0])

 === Timing for Test Problem ===
+---------+------------------------+
| problem | time                   |
+---------+------------------------+
| a       | 0.00022014000001036038 |
| b       | 0.00017433999994409533 |
| c       | 0.00016089000007468712 |
+---------+------------------------+


In [7]:
from logica import colab_logica

Could not import google.cloud.bigquery.
Could not import google.cloud.auth.
Could not import google.colab.widgets.


In [10]:
%%logica G,S

@AttachDatabase("db", "graphs.db");

N() = 100;
D() = 10;

GStep1(a, b, i:) distinct :-
  a = NaturalHash("a-" ++ ToString(i)) % N(),
  b = NaturalHash("b-" ++ ToString(i)) % N(),
  a != b,
  i in Range(ToInt64(N() * D() * 1.1));

I() = l[N() * D()] :-
  l = Array{ i -> i :- GStep1(i:) };

G(a, b) :-
  GStep1(a, b, i:),
  i < I();


@Ground(G5c, "db.G5c", copy_to_file: "graph_bench/g5c.csv");
G5c := G(N: 500);
@Ground(G6c, "db.G6c", copy_to_file: "graph_bench/g6c.csv");
G6c := G(N: 600);
@Ground(G7c, "db.G7c", copy_to_file: "graph_bench/g7c.csv");
G7c := G(N: 700);

@Ground(G1k, "db.G1k", copy_to_file: "graph_bench/g1k.csv");
G1k := G(N: 1000);
@Ground(G2k, "db.G2k", copy_to_file: "graph_bench/g2k.csv");
G2k := G(N: 2000);
@Ground(G3k, "db.G3k", copy_to_file: "graph_bench/g3k.csv");
G3k := G(N: 3000);
@Ground(G4k, "db.G4k", copy_to_file: "graph_bench/g4k.csv");
G4k := G(N: 4000);
@Ground(G5k, "db.G5k", copy_to_file: "graph_bench/g5k.csv");
G5k := G(N: 5000);

S() += 1 :- G5c() | G6c() | G7c() | G1k() | G2k() | G3k() | G4k() | G5k();
 


Query is stored at G_sql variable.
The following table is stored at G variable.


,col0,col1
0,95,60
1,80,92
2,32,61
3,29,66
4,73,4
5,61,71
6,46,11
7,80,32
8,10,51
9,85,25


Query is stored at S_sql variable.
The following table is stored at S variable.


,logica_value
0,168000


In [7]:
graphs = ['G1k', 'G2k', 'G3k', 'G4k', 'G5k']

In [9]:
%%logica Tree, NumNodes

@AttachDatabase("db", "graphs.db");

Depth() = 5;
DegreeRange(1, 5);

InnerTree(x, y) :-
    (x = "0" | InnerTree(something, x)),
    Length(x) < Depth(),
    DegreeRange(d1, d2),
    d = NaturalHash("d-" ++ x) % (d2 - d1) + d1,
    i in Range(d),
    y = x ++ ToString(i);

# Creating a non-recursive face, so we can customly ground it.
Tree(x, y) :- InnerTree(x, y);

@Ground(Tree5, "db.Tree5", copy_to_file: "graph_bench/tree5.csv");
Tree5 := Tree(Depth: 5);
@Ground(Tree6, "db.Tree6", copy_to_file: "graph_bench/tree6.csv");
Tree6 := Tree(Depth: 6);
@Ground(Tree7, "db.Tree7", copy_to_file: "graph_bench/tree7.csv");
Tree7 := Tree(Depth: 7);
@Ground(Tree8, "db.Tree8", copy_to_file: "graph_bench/tree8.csv");
Tree8 := Tree(Depth: 8);
@Ground(Tree9, "db.Tree9", copy_to_file: "graph_bench/tree9.csv");
Tree9 := Tree(Depth: 9);
@Ground(Tree10, "db.Tree10", copy_to_file: "graph_bench/tree10.csv");
Tree10 := Tree(Depth: 10);
@Ground(Tree11, "db.Tree11", copy_to_file: "graph_bench/tree11.csv");
Tree11 := Tree(Depth: 11);
@Ground(Tree12, "db.Tree12", copy_to_file: "graph_bench/tree12.csv");
Tree12 := Tree(Depth: 12);

NumNodes(tree5: Sum{ 1 :- Tree5()},
         tree6: Sum{ 1 :- Tree6()},
         tree7: Sum{ 1 :- Tree7()},
         tree8: Sum{ 1 :- Tree8()},
         tree9: Sum{ 1 :- Tree9()},
         tree10: Sum{ 1 :- Tree10()},
         tree11: Sum{ 1 :- Tree11()},
         tree12: Sum{ 1 :- Tree12()}
        );


Query is stored at Tree_sql variable.
The following table is stored at Tree variable.


,col0,col1
0,0,00
1,0,01
2,0110,01100
3,0110,01101
4,0110,01102
5,011,0110
6,011,0111
7,00,000
8,0111,01110
9,0111,01111


Query is stored at NumNodes_sql variable.
The following table is stored at NumNodes variable.


,tree5,tree6,tree7,tree8,tree9,tree10,tree11,tree12
0,25,68,180,471,1205,3080,7781,19504


In [13]:
trees = ['Tree7', 'Tree8', 'Tree9', 'Tree10', 'Tree11', 'Tree12']

In [14]:
%%loop ("Transitive Closure", graphs)
%%logica TC

@AttachDatabase("db", "graphs.db");

G(a, b) :- db.{loop_parameter}(a, b);

@Recursive(TC, ∞, stop: Done);
TC(a, b) distinct :- G(a, b);
TC(a, c) distinct :- TC(a, b), G(b, c);

PrevTC(a, b) :- TC(a, b);

Done() :- TC(a, b) => PrevTC(a, b);

Query is stored at TC_sql variable.
The following table is stored at TC variable.


,col0,col1
0,931,419
1,243,80
2,542,14
3,95,159
4,325,567
5,764,632
6,984,756
7,683,783
8,398,110
9,747,944


 
Query is stored at TC_sql variable.
The following table is stored at TC variable.


,col0,col1
0,571,1979
1,124,1595
2,999,1893
3,180,1194
4,1744,1179
5,1234,758
6,1627,489
7,1990,239
8,964,894
9,1089,885


 
Query is stored at TC_sql variable.
The following table is stored at TC variable.


...

 
Query is stored at TC_sql variable.
The following table is stored at TC variable.


,col0,col1
0,371,3577
1,3324,2395
2,1781,2812
3,2131,1370
4,3460,2456
5,1523,2905
6,3632,3018
7,3328,2277
8,1658,1025
9,348,2770


 
Query is stored at TC_sql variable.
The following table is stored at TC variable.


,col0,col1
0,4140,727
1,4906,4627
2,622,3981
3,686,3233
4,2964,1894
5,3544,3842
6,2554,1365
7,1831,1809
8,2434,4558
9,1868,3756


 
 === Timing for Transitive Closure ===
+---------+--------------------+
| problem | time               |
+---------+--------------------+
| G1k     | 1.8763943879999943 |
| G2k     | 3.617054680000024  |
| G3k     | 7.230493697999918  |
| G4k     | 12.564421521999975 |
| G5k     | 19.55763669199996  |
+---------+--------------------+


In [15]:
%%loop ("Pairwise Distances", graphs)
%%logica D

@AttachDatabase("db", "graphs.db");

G(a, b) :- db.{loop_parameter}(a, b);

@Recursive(D, ∞, stop: Done);
D(a, b) Min= 1 :- G(a, b);
D(a, c) Min= D(a, b) + 1 :- G(b, c);

PrevD(a, b) :- D(a, b);

Done() :- D(a, b) => PrevD(a, b);

Query is stored at D_sql variable.
The following table is stored at D variable.


,col0,col1,logica_value
0,471,966,1
1,399,731,1
2,969,313,1
3,630,953,1
4,41,215,1
5,861,679,1
6,558,186,1
7,472,89,1
8,416,15,1
9,706,999,1


 
Query is stored at D_sql variable.
The following table is stored at D variable.


,col0,col1,logica_value
0,136,843,1
1,1379,864,1
2,589,1751,1
3,784,859,1
4,1012,898,1
5,591,827,1
6,309,942,1
7,379,1439,1
8,1169,204,1
9,1433,1216,1


 
Query is stored at D_sql variable.
The following table is stored at D variable.


...

 
Query is stored at D_sql variable.
The following table is stored at D variable.


,col0,col1,logica_value
0,1826,1104,1
1,59,1395,1
2,2471,2966,1
3,3210,84,1
4,2774,995,1
5,3781,1142,1
6,2982,1367,1
7,1755,175,1
8,3812,507,1
9,2230,1068,1


 
Query is stored at D_sql variable.
The following table is stored at D variable.


,col0,col1,logica_value
0,2379,4010,1
1,781,967,1
2,1978,3870,1
3,4233,1699,1
4,1286,2476,1
5,1348,474,1
6,1916,3457,1
7,2125,3540,1
8,2381,2207,1
9,2707,2646,1


 
 === Timing for Pairwise Distances ===
+---------+--------------------+
| problem | time               |
+---------+--------------------+
| G1k     | 2.099346245999982  |
| G2k     | 4.338347781999914  |
| G3k     | 8.60555926699999   |
| G4k     | 15.257616865000045 |
| G5k     | 23.63516611        |
+---------+--------------------+


In [20]:
from logica.common import graph

graph.DirectedGraph(Tree)

In [17]:
%%loop ("Same Generation", trees)
%%logica SGCheck, SGGold


@AttachDatabase("db", "graphs.db");

Tree(x, y) :- db.{loop_parameter}(x, y);
#Tree(x, y) :- db.Tree12(x, y);

Node(x) distinct :- x in [a, b], Tree(a, b), Str(x);

# Tree("a", "b");
# Tree("a", "c");
# Tree("c", "d");

@Recursive(SG, ∞, stop: Done);
SG(x, y) distinct :- Tree(a, x), Tree(a, y);
SG(x, y) distinct :- SG(a, b), Tree(a, x), Tree(b, y);
PrevSG(x, y) :- SG(x, y);
Done() :- Sum{ 1 :- SG(x, y) } == Sum{ 1 :- PrevSG(x, y) };
    
SGCheck() += 1 :- SG();
SGGold() += 1 :- Node(a), Node(b), Length(a) == Length(b);


Query is stored at SGCheck_sql variable.
The following table is stored at SGCheck variable.


,logica_value
0,14656


Query is stored at SGGold_sql variable.
The following table is stored at SGGold variable.


,logica_value
0,14657


 
 
Query is stored at SGCheck_sql variable.
The following table is stored at SGCheck variable.


...

 
 
Query is stored at SGCheck_sql variable.
The following table is stored at SGCheck variable.


,logica_value
0,163681848


Query is stored at SGGold_sql variable.
The following table is stored at SGGold variable.


,logica_value
0,163681849


 
 
 === Timing for Same Generation ===
+---------+--------------------+
| problem | time               |
+---------+--------------------+
| Tree7   | 0.7857960700000604 |
| Tree8   | 0.9151512260000345 |
| Tree9   | 1.6111061390000714 |
| Tree10  | 2.1807925879999175 |
| Tree11  | 7.708316269999955  |
| Tree12  | 37.75424764900015  |
+---------+--------------------+


In [21]:
timing

{'a': 0.00022921100003259198,
 'b': 0.0001869599999508864,
 'c': 0.00017046899995420972,
 'G1k': 2.099346245999982,
 'G2k': 4.338347781999914,
 'G3k': 8.60555926699999,
 'G4k': 15.257616865000045,
 'G5k': 23.63516611,
 'Tree7': 0.7857960700000604,
 'Tree8': 0.9151512260000345,
 'Tree9': 1.6111061390000714,
 'Tree10': 2.1807925879999175,
 'Tree11': 7.708316269999955,
 'Tree12': 37.75424764900015}

In [22]:
timing

{'a': 0.00022921100003259198,
 'b': 0.0001869599999508864,
 'c': 0.00017046899995420972,
 'G1k': 2.099346245999982,
 'G2k': 4.338347781999914,
 'G3k': 8.60555926699999,
 'G4k': 15.257616865000045,
 'G5k': 23.63516611,
 'Tree7': 0.7857960700000604,
 'Tree8': 0.9151512260000345,
 'Tree9': 1.6111061390000714,
 'Tree10': 2.1807925879999175,
 'Tree11': 7.708316269999955,
 'Tree12': 37.75424764900015}

In [23]:
%%loop ("Same Generation Logica-style", trees)
%%logica SGCheck, SGGold


@AttachDatabase("db", "graphs.db");

Tree(x, y) :- db.{loop_parameter}(x, y);
#Tree(x, y) :- db.Tree7(x, y);

Node(x) distinct :- x in [a, b], Tree(a, b), Str(x);
# Tree("a", "b");
# Tree("a", "c");
# Tree("c", "d");


@Recursive(Gen, ∞, stop: Done);
Gen(x) Min= x :- Node(x);
NextGen(Gen(x)) Min= Gen(y) :- Tree(x, y);
Gen(y) Min= NextGen(Gen(x)) :- Tree(x, y);

PrevGen(x) = Gen(x);
Done() :- Gen(x) => Gen(x) = PrevGen(x);

SG(x, y) :- Gen(x) = Gen(y);
    

SGCheck() += 1 :- SG();
SGGold() += 1 :- Node(a), Node(b), Length(a) == Length(b);


Query is stored at SGCheck_sql variable.
The following table is stored at SGCheck variable.


,logica_value
0,14657


Query is stored at SGGold_sql variable.
The following table is stored at SGGold variable.


,logica_value
0,14657


 
 
Query is stored at SGCheck_sql variable.
The following table is stored at SGCheck variable.


...

 
 
Query is stored at SGCheck_sql variable.
The following table is stored at SGCheck variable.


,logica_value
0,163681849


Query is stored at SGGold_sql variable.
The following table is stored at SGGold variable.


,logica_value
0,163681849


 
 
 === Timing for Same Generation Logica-style ===
+---------+--------------------+
| problem | time               |
+---------+--------------------+
| Tree7   | 1.0253999930000646 |
| Tree8   | 1.1346653619998506 |
| Tree9   | 1.048337869000079  |
| Tree10  | 1.2038090660000762 |
| Tree11  | 1.4260789869999826 |
| Tree12  | 2.5498552100000325 |
+---------+--------------------+


In [16]:
timing

{'a': 0.00015937499999996163,
 'b': 0.0001060420000000839,
 'c': 9.687500000010729e-05,
 'G1k': 1.5715949169999988,
 'G2k': 4.012602374999993,
 'G3k': 7.555930625000002,
 'G4k': 17.507719375000008,
 'G5k': 27.752416333,
 'Tree7': 4.729736041999985,
 'Tree8': 0.8861973330000126,
 'Tree9': 0.9397843750000163,
 'Tree10': 0.9565629169999852,
 'Tree11': 1.3457253750000007,
 'Tree12': 2.0904798330000176}

In [24]:
for report in reports:
    print(report)

 === Timing for Test Problem ===
+---------+------------------------+
| problem | time                   |
+---------+------------------------+
| a       | 0.00022921100003259198 |
| b       | 0.0001869599999508864  |
| c       | 0.00017046899995420972 |
+---------+------------------------+
 === Timing for Transitive Closure ===
+---------+--------------------+
| problem | time               |
+---------+--------------------+
| G1k     | 0.8889665329999161 |
| G2k     | 0.6444625069999574 |
| G3k     | 0.691283713999951  |
| G4k     | 0.6476772670000628 |
| G5k     | 0.6453234560000283 |
+---------+--------------------+
 === Timing for Transitive Closure ===
+---------+--------------------+
| problem | time               |
+---------+--------------------+
| G1k     | 1.8763943879999943 |
| G2k     | 3.617054680000024  |
| G3k     | 7.230493697999918  |
| G4k     | 12.564421521999975 |
| G5k     | 19.55763669199996  |
+---------+--------------------+
 === Timing for Pairwise Distances =